In [148]:
# Training sets get 
# one training set for projections ((id + 1) mod 6 AND (id + 3) mod 6 every 15-month-period)
# one slightly overlapping training set for hyperparameters
# one dev set for projections
# one slightly overlapping dev set for hyperparameters

# alpha = how much we deviate from our current portfolio
# beta = threshold where we liquidate stocks. if it drops below b% of the average of our per-stock-investment for that asset class, we liquidate
# gamma = how many stocks we want at a time
# kappa = minimum allowed spread between asset classes (eg. if we have 5 tech stocks and kappa=2, we should have at minimum 3 commodity stocks)
# delta = TODO: each stock has some predicted range. this will always consider projected percentages to be delta% of that range
# todo: one more that checks the weight of each day, OR one that limits information to a certain day
# TODO: consider dividends in stocks


def calculate_adjustments(investments, alpha=0.50, beta=0.10, gamma=25, kappa=2, delta=0.4):    
    # Hyper parameter that determines weight of previous capital in readjustment

    # Calculate the total invested capital across all investments
    total_capital = sum(investment['invested_capital'] for investment in investments)

    # Find the minimum predicted change among all investments
    min_change = min(investment['predicted_change'] for investment in investments)
    # Calculate the adjusted total change by subtracting the minimum change from each predicted change
    total_change = sum(investment['predicted_change'] - min_change for investment in investments)
    
    new_investments = []
    for investment in investments:
        # Calculate new investment amount based on weighted average of normalized predicted change and proportion of total capital
        new_investment = (alpha * ((investment['predicted_change'] - min_change) / total_change) + (1-alpha) * (investment['invested_capital'] / total_capital)) * total_capital
        new_investments.append(new_investment)

    adjustments = []
    for i in range(len(new_investments)):
        # Calculate the difference between new and old investment amounts to find the adjustment
        adjustment = new_investments[i] - investments[i]['invested_capital']
        # Include the name of the investment with its corresponding adjustment
        adjustments.append((investments[i]['name'], adjustment))

    print(f"Portfolio adjustments: with risk of {alpha * 100:.0f}%", [f"{adj[0]} ++{adj[1]:.2f}" if adj[1] > 0 else f"{adj[0]} --{abs(adj[1]):.2f}" for adj in adjustments], "\n")


In [149]:
# TODO: add in expense ratio for ETF
investments = [
    {'name': 'TSLA', 'invested_capital': 5000, 'predicted_change': -0.07},
    {'name': 'WLMT', 'invested_capital': 5000, 'predicted_change': -0.05},
    {'name': 'BTC', 'invested_capital': 5000, 'predicted_change': 0.15},
    {'name': 'US DOLLARS', 'invested_capital': 5000, 'predicted_change': 0.00},
]

calculate_adjustments(investments, 0.05)
calculate_adjustments(investments, 0.50)
calculate_adjustments(investments, 0.95)

Portfolio adjustments: with risk of 5% ['TSLA --250.00', 'WLMT --185.48', 'BTC ++459.68', 'US DOLLARS --24.19'] 

Portfolio adjustments: with risk of 50% ['TSLA --2500.00', 'WLMT --1854.84', 'BTC ++4596.77', 'US DOLLARS --241.94'] 

Portfolio adjustments: with risk of 95% ['TSLA --4750.00', 'WLMT --3524.19', 'BTC ++8733.87', 'US DOLLARS --459.68'] 

